<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#init" data-toc-modified-id="init-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>init</a></span></li><li><span><a href="#get-html" data-toc-modified-id="get-html-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>get html</a></span></li><li><span><a href="#analy-html" data-toc-modified-id="analy-html-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>analy html</a></span></li><li><span><a href="#finaly" data-toc-modified-id="finaly-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>finaly</a></span></li></ul></div>

# init

**基本方針**
1. requestsというライブラリーでwebサイトのhtmlをgetする。
2. beautiful soupというライブラリーでgetしたhtmlを解析してデータ化する。

In [4]:
import requests
from bs4 import BeautifulSoup

In [3]:
# beautiful soupがなかったらこれでダウンロード
!pip install beautifulsoup4

# get html

In [5]:
url = "http://www.jma.go.jp/jp/amedas_h/yesterday-61286.html?areaCode=000&groupCode=44"

In [8]:
# url先のhtmlをget
response = requests.get(url)

In [9]:
# htmlとはこんなやつね
print(response.text)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">
<html lang="ja">
<head>
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
<meta name="Author" content="気象庁 Japan Meteorological Agency">
<meta name="keywords" content="気象庁 Japan Meteorological Agency">
<title>2020年07月07日　京都(キョウト)　毎正時の観測データ</title>
<style type="text/css" media="all"><!-- @import url(../common/default.css); --></style>
<style type="text/css" media="all"><!-- @import url(../common/weather_info_v2.css); --></style>
<style type="text/css" media="all"><!-- @import url(./css/amedastable.css); --></style>
<link href="../common/print.css" rel="stylesheet" type="text/css" media="print">
<script src="../common/cmnDefine.js" type="text/javascript"></script>
<script src="../common/cmnCtrl_v2.js" type="text/javascript"></script>
<script src="../common/weather_info.js" type="text/javascript"></script>
<script src="../common/contents_walker.js" type="text/javascript"></script>
<script src="../common/cont

In [10]:
html = response.text

In [11]:
"a" in "a", "b"

(True, 'b')

In [12]:
# 関数にしておくと楽
def get_html_kyoto(day):
    '''京都市の気象情報ページのhtmlを取得する関数
    '''
    if day not in ("today", "yesterday"): # 今日か昨日か指定せんかい！っていう記述
        print("todayかyesterdayを入力して下さい")
        return None
    url = "http://www.jma.go.jp/jp/amedas_h/%s-61286.html?areaCode=000&groupCode=44" % day # urlを作り出す
    response = requests.get(url)
    return response.text

In [13]:
# テスト
get_html_kyoto("today")

'<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">\n<html lang="ja">\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">\n<meta name="Author" content="気象庁 Japan Meteorological Agency">\n<meta name="keywords" content="気象庁 Japan Meteorological Agency">\n<title>2020年07月08日\u3000京都(キョウト)\u3000毎正時の観測データ</title>\n<style type="text/css" media="all"><!-- @import url(../common/default.css); --></style>\n<style type="text/css" media="all"><!-- @import url(../common/weather_info_v2.css); --></style>\n<style type="text/css" media="all"><!-- @import url(./css/amedastable.css); --></style>\n<link href="../common/print.css" rel="stylesheet" type="text/css" media="print">\n<script src="../common/cmnDefine.js" type="text/javascript"></script>\n<script src="../common/cmnCtrl_v2.js" type="text/javascript"></script>\n<script src="../common/weather_info.js" type="text/javascript"></script>\n<script src="../common/contents_walker.js" type="text/javascript"></script>\n<

# analy html

※ここでやりたいことは必要な部分だけ抜き出してくること

In [14]:
# 準備
html = get_html_kyoto("today")
soup = BeautifulSoup(html, "xml")

In [23]:
# tdタグかつclass="block middle"となってるところがデータだから、その部分だけぶっこぬく
values = soup.find_all("td", class_="block middle")

In [27]:
# 抜けました
for v in values:
    print(v.text)

24.1
0.5
東北東
0.8

95
1005.7
23.7
0.5
西北西
0.5

96
1005.0
23.9
4.5
南南東
0.8

97
1004.4
23.7
17.0
東
2.3
0.0
98
1002.5
23.4
27.0
北北東
3.9
0.0
98
1002.3
22.4
26.0
南南東
1.6
0.0
94
1004.2
22.1
2.0
南南西
3.1
0.0
95
1005.6
22.7
0.5
南
2.0
0.0
92
1005.6
23.8
0.0
南南東
2.4
0.0
86
1006.4
25.5
0.0
南東
0.7
0.2
76
1006.1




































































































In [34]:
# 綺麗に表形式にする
data = [v.text for v in values]
span = 7
names = ["気温[℃]", "降水量[mm]",
         "風向き[16方位]", "風速[m/s]",
         "日照時間[h]", "湿度[%]", "気圧[hPa]"]
dic = dict()
for i, name in enumerate(names):
    dic[name] = data[i::span]

In [36]:
import pandas as pd

In [38]:
df = pd.DataFrame(dic)

In [40]:
df

,気温[℃],降水量[mm],風向き[16方位],風速[m/s],日照時間[h],湿度[%],気圧[hPa]
0,24.1,0.5,東北東,0.8,,95,1005.7
1,23.7,0.5,西北西,0.5,,96,1005.0
2,23.9,4.5,南南東,0.8,,97,1004.4
3,23.7,17.0,東,2.3,0.0,98,1002.5
4,23.4,27.0,北北東,3.9,0.0,98,1002.3
5,22.4,26.0,南南東,1.6,0.0,94,1004.2
6,22.1,2.0,南南西,3.1,0.0,95,1005.6
7,22.7,0.5,南,2.0,0.0,92,1005.6
8,23.8,0.0,南南東,2.4,0.0,86,1006.4
9,25.5,0.0,南東,0.7,0.2,76,1006.1


In [41]:
# これも関数化しておく
def html2df(html):
    '''気象情報ページのhtmlをpandas.DataFrameに変換する関数
    '''
    soup = BeautifulSoup(html, "xml") # 準備
    values = soup.find_all("td", class_="block middle") # 検索
    
    # データ整形
    data = [v.text for v in values]
    span = 7
    names = ["気温[℃]", "降水量[mm]",
             "風向き[16方位]", "風速[m/s]",
             "日照時間[h]", "湿度[%]", "気圧[hPa]"]
    dic = dict()
    for i, name in enumerate(names):
        dic[name] = data[i::span]
    
    return pd.DataFrame(dic)

In [43]:
html2df(html)

,気温[℃],降水量[mm],風向き[16方位],風速[m/s],日照時間[h],湿度[%],気圧[hPa]
0,24.1,0.5,東北東,0.8,,95,1005.7
1,23.7,0.5,西北西,0.5,,96,1005.0
2,23.9,4.5,南南東,0.8,,97,1004.4
3,23.7,17.0,東,2.3,0.0,98,1002.5
4,23.4,27.0,北北東,3.9,0.0,98,1002.3
5,22.4,26.0,南南東,1.6,0.0,94,1004.2
6,22.1,2.0,南南西,3.1,0.0,95,1005.6
7,22.7,0.5,南,2.0,0.0,92,1005.6
8,23.8,0.0,南南東,2.4,0.0,86,1006.4
9,25.5,0.0,南東,0.7,0.2,76,1006.1


# finaly

In [45]:
# 結局ここだけコピペして！
import pandas as pd
import requests
from bs4 import BeautifulSoup

def get_html_kyoto(day):
    '''京都市の気象情報ページのhtmlを取得する関数
    '''
    if day not in ("today", "yesterday"): # 今日か昨日か指定せんかい！っていう記述
        print("todayかyesterdayを入力して下さい")
        return None
    url = "http://www.jma.go.jp/jp/amedas_h/%s-61286.html?areaCode=000&groupCode=44" % day # urlを作り出す
    response = requests.get(url)
    return response.text

def html2df(html):
    '''気象情報ページのhtmlをpandas.DataFrameに変換する関数
    '''
    soup = BeautifulSoup(html, "xml") # 準備
    values = soup.find_all("td", class_="block middle") # 検索
    
    # データ整形
    data = [v.text for v in values]
    span = 7
    names = ["気温[℃]", "降水量[mm]",
             "風向き[16方位]", "風速[m/s]",
             "日照時間[h]", "湿度[%]", "気圧[hPa]"]
    dic = dict()
    for i, name in enumerate(names):
        dic[name] = data[i::span]
    
    return pd.DataFrame(dic)

# 2つとも一気に処理する
def get_amadas_kyoto(day):
    
    html = get_html_kyoto(day)
    
    if html is None:
        print("todayかyesterdayを入力して下さい")
        return None

    return html2df(html)

In [46]:
get_amadas_kyoto("today")

,気温[℃],降水量[mm],風向き[16方位],風速[m/s],日照時間[h],湿度[%],気圧[hPa]
0,24.1,0.5,東北東,0.8,,95,1005.7
1,23.7,0.5,西北西,0.5,,96,1005.0
2,23.9,4.5,南南東,0.8,,97,1004.4
3,23.7,17.0,東,2.3,0.0,98,1002.5
4,23.4,27.0,北北東,3.9,0.0,98,1002.3
5,22.4,26.0,南南東,1.6,0.0,94,1004.2
6,22.1,2.0,南南西,3.1,0.0,95,1005.6
7,22.7,0.5,南,2.0,0.0,92,1005.6
8,23.8,0.0,南南東,2.4,0.0,86,1006.4
9,25.5,0.0,南東,0.7,0.2,76,1006.1


In [47]:
get_amadas_kyoto("yesterday")

,気温[℃],降水量[mm],風向き[16方位],風速[m/s],日照時間[h],湿度[%],気圧[hPa]
0,26.3,0.0,南,2.2,,88,1007.4
1,26.8,0.0,南南西,4.9,,85,1006.6
2,26.7,0.0,南南西,3.9,,86,1006.5
3,26.2,0.0,南,1.6,0.0,90,1006.8
4,25.9,0.0,東南東,2.0,0.0,91,1006.7
5,26.0,0.0,東北東,2.0,0.0,91,1007.1
6,25.9,0.0,南東,1.2,0.0,92,1007.1
7,26.2,0.5,南,2.5,0.0,89,1007.4
8,26.9,0.5,南南西,5.5,0.0,83,1007.3
9,28.1,0.0,南西,5.4,0.0,78,1007.2
